# 사용할 라이브러리 import

In [1]:
# MySQL 서버 연결
from sqlalchemy import create_engine
import mysql.connector

# MySQL 서버 config 파싱
import configparser

import numpy as np
import pandas as pd
from collections import Counter,defaultdict
from itertools import chain

import json
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import configparser

# config parser 객체 생성
config = configparser.ConfigParser()

# parser객체로 config.ini 파일 읽기
config.read('config.ini')

# 변수저장
'''
user = config['DEFAULT']['ADMIN_USER_NAME']
passwd = config['DEFAULT']['ADMIN_PASSWORD']
host = config['DEFAULT']['RDS_ENDPOINT']
port = config['DEFAULT']['PORT']
database = config['DEFAULT']['DEFAULT_DATABASE']
'''

user = 'root'
passwd = 'wilfriedzaha11!'
host = '127.0.0.1'
port = '3306'
database = 'melon_recom_db'

In [3]:
melon_recom_db = mysql.connector.connect(
    host = host,
    user = user,
    password = passwd,
    database = database,
    port = port,
)

In [4]:
melon_cursor = melon_recom_db.cursor()

# 테이블 생성

## 플레이리스트 테이블 생성

In [5]:
query = '''create table `playlist` 
                        (`playlist_id` INTEGER PRIMARY KEY, 
                         `playlist_title` TEXT,
                         `update_date` DATETIME, 
                         `like_cnt` INTEGER)'''

melon_cursor.execute(query)

## 태그 테이블 생성

In [6]:
query = '''create table `tag` 
                        (`tag_id` INTEGER PRIMARY KEY, 
                         `tag` TEXT) '''

melon_cursor.execute(query)

## 플레이리스트_태그 관계테이블 생성

In [7]:
query = '''create table `playlist_tag` 
                        (`tag_id` INTEGER, 
                         `playlist_id` INTEGER,
                         FOREIGN KEY(tag_id) REFERENCES tag(tag_id) on delete cascade,
                         FOREIGN KEY(playlist_id) REFERENCES playlist(playlist_id) on delete cascade)'''

melon_cursor.execute(query)

## 가수정보 테이블

In [8]:
query = '''create table `artist` 
                        (`artist_id` INTEGER PRIMARY KEY, 
                         `artist_name` TEXT,
                         `artist_main_genre` TEXT)'''

melon_cursor.execute(query)

## 가수 플레이리스트 관계 테이블

In [9]:
query = '''create table `artist_playlist` 
                        (`playlist_id` INTEGER , 
                         `artist_id` INTEGER,
                         FOREIGN KEY(artist_id) REFERENCES artist(artist_id) on delete cascade,
                         FOREIGN KEY(playlist_id) REFERENCES playlist(playlist_id) on delete cascade)'''

melon_cursor.execute(query)

## 노래 정보 테이블

In [10]:
query = '''create table `song` 
                        (`song_id` INTEGER PRIMARY KEY, 
                         `issue_date` TEXT,
                         `album_name` TEXT,
                         `album_id` INTEGER,
                         `song_name` TEXT,
                         `added_cnt` INTEGER,
                         `thumb_url` TEXT)'''

melon_cursor.execute(query)

## 가수 노래 관계 테이블

In [11]:
query = '''create table `artist_song` 
                        (`song_id` INTEGER , 
                         `artist_id` INTEGER,
                         FOREIGN KEY(artist_id) REFERENCES artist(artist_id) on delete cascade,
                         FOREIGN KEY(song_id) REFERENCES song(song_id) on delete cascade)'''

melon_cursor.execute(query)

## 플레이리스트 노래 관계 테이블

In [12]:
query = '''create table `playlist_song` 
                        (`playlist_id` INTEGER,
                         `song_id` INTEGER ,
                         FOREIGN KEY(song_id) REFERENCES song(song_id) on delete cascade,
                         FOREIGN KEY(playlist_id) REFERENCES playlist(playlist_id) on delete cascade)'''

melon_cursor.execute(query)

## 대장르 테이블 

In [13]:
query = '''create table `genre_big` 
                        (`genre_big_id` INTEGER PRIMARY KEY, 
                         `genre_big_code` TEXT,
                         `genre_name` TEXT)'''

melon_cursor.execute(query)

## 대장르 노래 관계 테이블

In [14]:
query = '''create table `genre_big_song` 
                        (`genre_big_id` INTEGER,
                         `song_id` INTEGER ,
                         FOREIGN KEY(song_id) REFERENCES song(song_id) on delete cascade,
                         FOREIGN KEY(genre_big_id) REFERENCES genre_big(genre_big_id) on delete cascade)'''

melon_cursor.execute(query)

## 세부장르 테이블

In [15]:
query = '''create table `genre_detail` 
                        (`genre_detail_id` INTEGER PRIMARY KEY, 
                         `genre_detail_code` TEXT,
                         `genre_name` TEXT)'''

melon_cursor.execute(query)

## 세부장르 노래 관계 테이블

In [16]:
query = '''create table `genre_detail_song` 
                        (`genre_detail_id` INTEGER,
                         `song_id` INTEGER ,
                         FOREIGN KEY(song_id) REFERENCES song(song_id) on delete cascade,
                         FOREIGN KEY(genre_detail_id) REFERENCES genre_detail(genre_detail_id) on delete cascade)'''

melon_cursor.execute(query)

# 데이터 삽입

In [17]:
# MySQL 서버 연결
engine = create_engine(f"mysql+pymysql://{user}:{passwd}@{host}:{port}/{database}",encoding='utf-8')
conn = engine.connect()

## 데이터 불러오기 

In [18]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
with open('data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
with open('data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)
    
# 만들어 놓은 csv파일을 이용해서 dataframe 생성
artist_df = pd.read_csv('./data/artist_table.csv',encoding='utf-8')
artist_df.head(5)
    
song_artist_df = pd.read_csv('./data/song_artist.csv',encoding='utf-8')
song_artist_df.head(5)
    
playlist_artist_df = pd.read_csv('./data/playlist_artist.csv',encoding='utf-8')
playlist_artist_df.head(5)

train_df = pd.DataFrame.from_dict(train_dict)
song_df = pd.DataFrame.from_dict(song_dict)

In [19]:
# 테이블 생성 및 데이터 insert 하는 함수
def insert_data(df,table_name,connection):
    df.to_sql(name=table_name,con=connection,if_exists='append',index=False)

## 플레이리스트 테이블 insert

In [20]:
# index로 새로 플레이리스트 id 지정
train_df['playlist_id'] = train_df.index

# update 날짜는 datetime 형식으로 바꿈
train_df['updt_date'] = pd.to_datetime(train_df['updt_date'])

In [21]:
# SQL에 집어넣을 컬럼만 가지고 새 df 생성 
playlist_df = train_df[['playlist_id','plylst_title','updt_date','like_cnt']]

# 컬럼명 변경
playlist_df.columns = ['playlist_id','playlist_title','update_date','like_cnt']
playlist_df.head(5)

,playlist_id,playlist_title,update_date,like_cnt
0,0,여행같은 음악,2013-12-19 18:36:19,71
1,1,요즘 너 말야,2014-12-02 16:19:42,1
2,2,"편하게, 잔잔하게 들을 수 있는 곡.-",2017-08-28 07:09:34,17
3,3,크리스마스 분위기에 흠뻑 취하고 싶을때,2019-12-05 15:15:18,33
4,4,추억의 노래 ㅋ,2011-10-25 13:54:56,9


In [22]:
# 테이블 생성 및 데이터 insert
insert_data(playlist_df,'playlist',conn)

## 태그 테이블 insert

In [23]:
# train 데이터에 들어있는 tag들을 추출
tags = train_df['tags'].tolist()

# 1차원 리스트로 변경
tags = chain.from_iterable(tags)

In [24]:
# 중복태그 제거
unique_tags = list(set(tags))

In [25]:
# 태그별 id 붙여서 dataframe 생성
tags_df = pd.DataFrame(zip(range(len(unique_tags)),unique_tags),columns=['tag_id','tag'])
tags_df.head(5)

,tag_id,tag
0,0,걸그룹히트곡
1,1,
2,2,붐바야
3,3,잔잔한힙합
4,4,2026년


In [26]:
# 테이블 생성 및 데이터 insert
insert_data(tags_df,'tag',conn)

## 플레이리스트 태그 관계 테이블 insert

In [27]:
# 태그id와 태그명 간 변환 할수있게 dictionary 생성

id_to_tag_dict = dict(zip(range(len(unique_tags)),unique_tags))
tag_to_id_dict = dict(zip(unique_tags,range(len(unique_tags))))

In [28]:
# tag들만 추출 (2차원, 중복포함)
tags = train_df['tags'].tolist()

# tag id와 playlist id가 쌍(tuple)으로 들어있는 list 생성
tag_id_pid_list = []

# 추가
for tags,playlist in zip(tags,range(len(train_df))):
    for tag in tags:
        tag_id_pid_list.append((tag_to_id_dict[tag],playlist))

In [29]:
# 생성된 리스트를 가지고 dataframe 만들기
tag_playlist_df = pd.DataFrame(tag_id_pid_list)
tag_playlist_df.columns = ['tag_id','playlist_id']
tag_playlist_df.head(5)

,tag_id,playlist_id
0,3357,0
1,2898,1
2,7934,1
3,21996,2
4,1768,2


In [30]:
insert_data(tag_playlist_df,'playlist_tag',conn)

## 가수 정보 테이블

In [31]:
artist_df.head(5)

,artist_id,artist_name,artist_main_genre
0,0,unknown,GN1100
1,1,A.R.D.I.,GN1100
2,2,Al Kasha,GN1500
3,3,Benny Benassi,GN1100
4,4,Black Pata,GN1100


In [32]:
insert_data(artist_df,'artist',conn)

## 노래 테이블

### 모든 노래 id에 대해서 추가된 횟수를 0으로 초기화 한 다음에 Counter의 값을 넣어줌

In [33]:
added_song_list = list(chain.from_iterable(train_df['songs'].tolist()))
added_song_count = dict(Counter(added_song_list))
added_song_cnt = {}

for i in range(len(song_df)):
    added_song_cnt[i] = 0
for k,v in added_song_count.items():
    added_song_cnt[k] = v

In [34]:
# 노래당 플레이리스트에 추가된 횟수 추가
song_df['added_cnt'] = song_df['id'].map(lambda x : added_song_cnt[x])

# 썸네일 url은 비워둔 상태로
song_df['thumb_url'] = ''

# 필요한 컬럼만 가지고 새로운 dataframe 생성
song_df_for_table = song_df[['id','issue_date','album_name','album_id','song_name','added_cnt','thumb_url']]

# 컬럼명 변경
song_df_for_table.columns = ['song_id','issue_date','album_name','album_id','song_name','added_cnt','thumb_url']
song_df_for_table.head(5)

,song_id,issue_date,album_name,album_id,song_name,added_cnt,thumb_url
0,0,20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,Feelings,2,
1,1,20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,"Bach : Partita No. 4 In D Major, BWV 828 - II....",0,
2,2,20180518,Hit,4698747,Solsbury Hill (Remastered 2002),0,
3,3,20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,Feeling Right (Everything Is Nice) (Feat. Popc...,8,
4,4,20110824,그남자 그여자,2008470,그남자 그여자,2,


In [36]:
insert_data(song_df_for_table,'song',conn)

## 플레이리스트 노래 관계 테이블

In [37]:
train_df.head(5)

,tags,id,plylst_title,songs,like_cnt,updt_date,playlist_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19,0
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42,1
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34,2
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18,3
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56,4


In [38]:
# 노래 컬럼만 추출
songs_list = train_df['songs'].tolist()

# 플레이리스트 id와 노래들을 쌍(tuple)으로 갖는 list 생성
id_song_list = []

# 반복횟수와 playlist_id가 일치해서 enumerate 씀
for id,songs in enumerate(songs_list):
    for song in songs:
        id_song_list.append((id,song))

# 데이터 프레임 생성
song_playlist_df = pd.DataFrame(id_song_list,columns=['playlist_id','song_id'])
song_playlist_df.head(5)

,playlist_id,song_id
0,0,525514
1,0,129701
2,0,383374
3,0,562083
4,0,297861


In [39]:
insert_data(song_playlist_df,'playlist_song',conn)

## 대장르 테이블

In [40]:
genre_dict['GN9000'] = '기타장르'

genre_big = {}

# 모든 장르 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨 뒤 두자리가 00이면 대장류로 분류
    if k[-2:] == '00':
        
        # 맨앞 네자리를 키로 하는 대장류 딕셔너리 값 추가
        genre_big[k[:4]] = v

genre_big_list = []

for id,name in genre_big.items():
    genre_big_list.append((id+'00',name))
    
genre_big_df = pd.DataFrame(genre_big_list,columns=['genre_big_code','genre_name'])
genre_big_df['genre_big_id'] = genre_big_df.index
genre_big_df.head()

,genre_big_code,genre_name,genre_big_id
0,GN0100,발라드,0
1,GN0200,댄스,1
2,GN0300,랩/힙합,2
3,GN0400,R&B/Soul,3
4,GN0500,인디음악,4


In [41]:
insert_data(genre_big_df,'genre_big',conn)

## 대장르 노래 관계 테이블

In [42]:
genre_code_to_genre_id = dict(zip(genre_big_df.genre_big_code,genre_big_df.genre_big_id))

In [43]:
big_genre = song_df['song_gn_gnr_basket'].tolist()

song_id_genre_big_list = []

for song_id,genre_codes in enumerate(big_genre):
    for code in genre_codes:
        song_id_genre_big_list.append((song_id,genre_code_to_genre_id[code]))

song_id_genre_big_df = pd.DataFrame(song_id_genre_big_list,columns=['song_id','genre_big_id'])
song_id_genre_big_df.head(5)

,song_id,genre_big_id
0,0,8
1,1,15
2,2,8
3,3,10
4,4,17


In [44]:
insert_data(song_id_genre_big_df,'genre_big_song',conn)

## 세부장르 테이블

In [46]:
genre_detail_dict = {}

# 모든 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨뒤 두자리가 00이 아니면 대장류가 아닌거임!
    if k[-2:] != '00':
        
        # 그럴떈 아까만든 대장르 딕셔너리의 대장류 이름을 추가해서 이름을 수정해서 다시 넣어줌
        new_value = genre_big[k[:4]]+'_'+v
        genre_detail_dict[k] = new_value

In [49]:
genre_detail_list = []

for k,v in genre_detail_dict.items():
    genre_detail_list.append((k,v))


genre_detail_df = pd.DataFrame(genre_detail_list,columns=['genre_detail_code','genre_name'])
genre_detail_df['genre_detail_id'] = genre_detail_df.index
genre_detail_df.head(5)

,genre_detail_code,genre_name,genre_detail_id
0,GN0101,발라드_세부장르전체,0
1,GN0102,발라드_'80,1
2,GN0103,발라드_'90,2
3,GN0104,발라드_'00,3
4,GN0105,발라드_'10-,4


In [50]:
insert_data(genre_detail_df,'genre_detail',conn)

## 세부장르 노래 관계 테이블

In [52]:
detail_genre_code_to_genre_id = dict(zip(genre_detail_df.genre_detail_code,genre_detail_df.genre_detail_id))

In [54]:
detail_genre = song_df['song_gn_dtl_gnr_basket'].tolist()

song_id_genre_detail_list = []

for song_id,genre_codes in enumerate(detail_genre):
    for code in genre_codes:
        song_id_genre_detail_list.append((song_id,detail_genre_code_to_genre_id[code]))

song_id_genre_detail_df = pd.DataFrame(song_id_genre_detail_list,columns=['song_id','genre_detail_id'])
song_id_genre_detail_df.head(5)

,song_id,genre_detail_id
0,0,48
1,1,113
2,1,118
3,2,48
4,3,71


In [55]:
insert_data(song_id_genre_detail_df,'genre_detail_song',conn)